
# Taller Práctico 3: Procesamiento de video (CPU vs GPU CUDA)

Este notebook convierte un video en color a escala de grises usando:

- Un algoritmo **secuencial en CPU**.
- Un algoritmo **paralelo en GPU (CUDA con Numba)**.

Al final, calcula el **speedup** entre ambos enfoques.


In [1]:

# =============================================
# 1. Instalación de dependencias y verificación
# =============================================

!pip install -q opencv-python numba

# Verificar que Colab tiene GPU disponible
!nvidia-smi || echo "No hay GPU disponible"


Wed Dec  3 16:48:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:

# =============================================
# 2. Importaciones y configuración general
# =============================================

import cv2
import os
import time
import numpy as np

try:
    from numba import cuda
    cuda_disponible = cuda.is_available()
except Exception:
    cuda_disponible = False

print("CUDA disponible:", cuda_disponible)

# Ruta del video a procesar (sube tu archivo y ajusta el nombre)
VIDEO_PATH = "/content/video.mp4"  # Cambia el nombre si tu archivo se llama distinto


CUDA disponible: True


In [4]:

# =============================================
# 3. Función para extraer frames del video
# =============================================

def extraer_frames(video_path, carpeta_salida):
    os.makedirs(carpeta_salida, exist_ok=True)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"FPS del video: {fps}")
    print(f"Frames totales reportados: {total_frames}")

    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(os.path.join(carpeta_salida, f"frame_{i:04d}.jpg"), frame)
        i += 1

    cap.release()
    print(f"Frames extraídos en '{carpeta_salida}':", i)
    return fps, i

# Ejemplo de uso para CPU y GPU (comparten los mismos frames de entrada)
FPS_VIDEO, NUM_FRAMES = extraer_frames(VIDEO_PATH, "frames_color")


FPS del video: 24.0
Frames totales reportados: 96
Frames extraídos en 'frames_color': 96


In [5]:

# =============================================
# 4. Procesamiento secuencial en CPU
# =============================================

def procesar_cpu(carpeta_entrada, carpeta_salida):
    os.makedirs(carpeta_salida, exist_ok=True)

    archivos = sorted(os.listdir(carpeta_entrada))
    inicio = time.time()

    for file in archivos:
        ruta = os.path.join(carpeta_entrada, file)
        img = cv2.imread(ruta)
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(os.path.join(carpeta_salida, file), gray)

    fin = time.time()
    return fin - inicio

tiempo_cpu = procesar_cpu("frames_color", "frames_gray_cpu")
print("Tiempo total CPU (secuencial):", tiempo_cpu, "segundos")


Tiempo total CPU (secuencial): 0.16124510765075684 segundos


In [6]:

# =============================================
# 5. Procesamiento en GPU con CUDA (Numba)
# =============================================

if not cuda_disponible:
    print("CUDA no está disponible. Esta sección no se podrá ejecutar.")
else:
    from numba import cuda

    @cuda.jit
    def rgb_to_gray_cuda(img_color, img_gray):
        x, y = cuda.grid(2)
        if x < img_color.shape[0] and y < img_color.shape[1]:
            b = img_color[x, y, 0]
            g = img_color[x, y, 1]
            r = img_color[x, y, 2]
            gray_value = 0.114 * b + 0.587 * g + 0.299 * r
            img_gray[x, y] = gray_value

    def procesar_gpu(carpeta_entrada, carpeta_salida):
        os.makedirs(carpeta_salida, exist_ok=True)
        archivos = sorted(os.listdir(carpeta_entrada))

        inicio = time.time()

        for file in archivos:
            ruta = os.path.join(carpeta_entrada, file)
            img = cv2.imread(ruta)
            if img is None:
                continue

            img_np = img.astype(np.float32)
            gray_np = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)

            threadsperblock = (16, 16)
            blockspergrid_x = (img.shape[0] + threadsperblock[0] - 1) // threadsperblock[0]
            blockspergrid_y = (img.shape[1] + threadsperblock[1] - 1) // threadsperblock[1]

            rgb_to_gray_cuda[(blockspergrid_x, blockspergrid_y), threadsperblock](img_np, gray_np)

            cv2.imwrite(os.path.join(carpeta_salida, file), gray_np.astype("uint8"))

        fin = time.time()
        return fin - inicio

    tiempo_gpu = procesar_gpu("frames_color", "frames_gray_gpu")
    print("Tiempo total GPU (CUDA):", tiempo_gpu, "segundos")


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:937: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Tiempo total GPU (CUDA): 1.6563496589660645 segundos


In [7]:

# =============================================
# 6. Reconstrucción de video desde frames
# =============================================

def reconstruir_video(carpeta_frames, salida_video, fps, en_grises=True):
    archivos = sorted(os.listdir(carpeta_frames))
    if not archivos:
        raise RuntimeError(f"No hay frames en la carpeta: {carpeta_frames}")

    ejemplo = cv2.imread(os.path.join(carpeta_frames, archivos[0]), cv2.IMREAD_GRAYSCALE if en_grises else cv2.IMREAD_COLOR)
    if ejemplo is None:
        raise RuntimeError("No se pudo leer el frame de ejemplo.")

    h, w = ejemplo.shape if en_grises else ejemplo.shape[:2]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(salida_video, fourcc, fps, (w, h), not en_grises)

    for file in archivos:
        ruta = os.path.join(carpeta_frames, file)
        if en_grises:
            img = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
        else:
            img = cv2.imread(ruta)
        if img is None:
            continue
        out.write(img)

    out.release()
    print("Video generado:", salida_video)

# Reconstruir video para CPU
reconstruir_video("frames_gray_cpu", "/content/video_gray_cpu.mp4", FPS_VIDEO, en_grises=True)

# Reconstruir video para GPU (si se ejecutó)
if cuda_disponible and os.path.exists("frames_gray_gpu"):
    reconstruir_video("frames_gray_gpu", "/content/video_gray_gpu.mp4", FPS_VIDEO, en_grises=True)


Video generado: /content/video_gray_cpu.mp4
Video generado: /content/video_gray_gpu.mp4


In [8]:

# =============================================
# 7. Cálculo de speedup
# =============================================

if cuda_disponible and 'tiempo_gpu' in globals():
    speedup = tiempo_cpu / tiempo_gpu
    print("Tiempo CPU :", tiempo_cpu, "segundos")
    print("Tiempo GPU :", tiempo_gpu, "segundos")
    print("SPEEDUP (CPU / GPU) =", speedup)
else:
    print("No se pudo calcular el speedup porque no se ejecutó la versión GPU.")


Tiempo CPU : 0.16124510765075684 segundos
Tiempo GPU : 1.6563496589660645 segundos
SPEEDUP (CPU / GPU) = 0.09734967902333504


In [9]:

# =============================================
# 8. Descarga de videos desde Colab (opcional)
# =============================================

from google.colab import files

files.download("/content/video_gray_cpu.mp4")
files.download("/content/video_gray_gpu.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>